In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="4" 

In [2]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable

from lib.utils.config_parse import cfg_from_file
from lib.ssds_train import test_model, train_model

from lib.dataset.dataset_factory import load_data
from lib.utils.config_parse import cfg
from lib.modeling.model_builder import create_model

from lib.layers import Detect
from lib.utils.timer import Timer

import sys
import pickle
import numpy as np



config_file =  './experiments/cfgs/ssd_resnet18_test_voc_cp3_compressed.yml'
cfg_from_file(config_file)

print(config_file)

./experiments/cfgs/ssd_resnet18_test_voc_cp3_compressed.yml


In [3]:
 # Load data
print('===> Loading data')
train_loader = load_data(cfg.DATASET, 'train') if 'train' in cfg.PHASE else None
eval_loader = load_data(cfg.DATASET, 'eval') if 'eval' in cfg.PHASE else None
test_loader = load_data(cfg.DATASET, 'test') if 'test' in cfg.PHASE else None

===> Loading data


In [4]:
cfg.MODEL, cfg.PHASE

({'NETS': 'resnet_18',
  'SSDS': 'ssd',
  'HALF_PRECISION': True,
  'IMAGE_SIZE': [300, 300],
  'NUM_CLASSES': 21,
  'FEATURE_LAYER': [[7, 9, 'S', 'S', '', ''], [128, 256, 512, 256, 256, 128]],
  'STEPS': [[8, 8], [16, 16], [32, 32], [64, 64], [100, 100], [300, 300]],
  'SIZES': [[30, 30],
   [60, 60],
   [111, 111],
   [162, 162],
   [213, 213],
   [264, 264],
   [315, 315]],
  'ASPECT_RATIOS': [[1, 2, 3],
   [1, 2, 3],
   [1, 2, 3],
   [1, 2, 3],
   [1, 2],
   [1, 2]],
  'CLIP': True,
  'NUM_FUSED': 3},
 ['test'])

In [5]:
# Build model
print('===> Building model')
model, priorbox = create_model(cfg.MODEL)
with torch.no_grad():
    priors = Variable(priorbox.forward())
    detector = Detect(cfg.POST_PROCESS, priors)

===> Building model
==>Feature map size:
[(38, 38), (19, 19), (10, 10), (5, 5), (3, 3), (1, 1)]


/workspace/home/jgusak/ssds.pytorch/lib/layers/modules/l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)


In [7]:
# state_dict_path = "/workspace/raid/data/jgusak/ssds.pytorch/experiments/models/ssd_resnet18_voc/ssd_resnet_18_voc_epoch_175.pth"
state_dict_path = "/workspace/raid/data/jgusak/ssds.pytorch/experiments/models/ssd_resnet18_voc_cp3_compressed/ssd_resnet_18_voc_epoch_195.pth"
state_dict = torch.load(state_dict_path)


model_dict = model.state_dict()
pretrained_dict = {k: v for k, v in state_dict.items() 
                   if k in model_dict}

model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [8]:
# Utilize GPUs for computation
use_gpu = torch.cuda.is_available()
if use_gpu:
    print('Utilize GPUs for computation')
    print('Number of GPU available', torch.cuda.device_count())
    model.cuda()
    priors.cuda()
    cudnn.benchmark = True

Utilize GPUs for computation
Number of GPU available 1


In [9]:
output_dir = cfg.EXP_DIR
output_dir

'/workspace/raid/data/jgusak/ssds.pytorch/experiments/models/ssd_resnet18_voc_cp3_compressed'

In [10]:
def test_epoch(model, data_loader, detector, output_dir, use_gpu):
        model.eval()

        dataset = data_loader.dataset
        num_images = len(dataset)
        num_classes = detector.num_classes
        all_boxes = [[[] for _ in range(num_images)] for _ in range(num_classes)]
        empty_array = np.transpose(np.array([[],[],[],[],[]]),(1,0))

        _t = Timer()

        with torch.no_grad():
            for i in iter(range((num_images))):
                img = dataset.pull_image(i)
                scale = [img.shape[1], img.shape[0], img.shape[1], img.shape[0]]
                if use_gpu:
                    images = Variable(dataset.preproc(img)[0].unsqueeze(0).cuda())
                else:
                    images = Variable(dataset.preproc(img)[0].unsqueeze(0))
                _t.tic()
                # forward
                out = model(images, phase='eval')

                # detect
                detections = detector.forward(out)

                time = _t.toc()

                # TODO: make it smart:
                for j in range(1, num_classes):
                    cls_dets = list()
                    for det in detections[0][j]:
                        if det[0] > 0:
                            d = det.cpu().numpy()
                            score, box = d[0], d[1:]
                            box *= scale
                            box = np.append(box, score)
                            cls_dets.append(box)
                    if len(cls_dets) == 0:
                        cls_dets = empty_array
                    all_boxes[j][i] = np.array(cls_dets)

                # log per iter
                log = '\r==>Test: || {iters:d}/{epoch_size:d} in {time:.3f}s [{prograss}]\r'.format(
                        prograss='#'*int(round(10*i/num_images)) + '-'*int(round(10*(1-i/num_images))), iters=i, epoch_size=num_images,
                        time=time)
                sys.stdout.write(log)
                sys.stdout.flush()

        # write result to pkl
        with open(os.path.join(output_dir, 'detections.pkl'), 'wb') as f:
            pickle.dump(all_boxes, f, pickle.HIGHEST_PROTOCOL)

        # currently the COCO dataset do not return the mean ap or ap 0.5:0.95 values
        print('Evaluating detections')
        data_loader.dataset.evaluate_detections(all_boxes, output_dir)

In [10]:
test_epoch(model, test_loader, detector, output_dir, use_gpu)

Evaluating detections in 0.347s [##########]
Writing aeroplane VOC results file
Writing bicycle VOC results file
Writing bird VOC results file
Writing boat VOC results file
Writing bottle VOC results file
Writing bus VOC results file
Writing car VOC results file
Writing cat VOC results file
Writing chair VOC results file
Writing cow VOC results file
Writing diningtable VOC results file
Writing dog VOC results file
Writing horse VOC results file
Writing motorbike VOC results file
Writing person VOC results file
Writing pottedplant VOC results file
Writing sheep VOC results file
Writing sofa VOC results file
Writing train VOC results file
Writing tvmonitor VOC results file
VOC07 metric? Yes
AP for aeroplane = 0.6850
AP for bicycle = 0.7139
AP for bird = 0.5656
AP for boat = 0.5246
AP for bottle = 0.3112
AP for bus = 0.7477
AP for car = 0.7710
AP for cat = 0.7995
AP for chair = 0.4454
AP for cow = 0.6347
AP for diningtable = 0.5909
AP for dog = 0.7234
AP for horse = 0.7802
AP for motorbik

In [11]:
test_epoch(model, test_loader, detector, output_dir, use_gpu)

Evaluating detections in 0.270s [##########]
Writing aeroplane VOC results file
Writing bicycle VOC results file
Writing bird VOC results file
Writing boat VOC results file
Writing bottle VOC results file
Writing bus VOC results file
Writing car VOC results file
Writing cat VOC results file
Writing chair VOC results file
Writing cow VOC results file
Writing diningtable VOC results file
Writing dog VOC results file
Writing horse VOC results file
Writing motorbike VOC results file
Writing person VOC results file
Writing pottedplant VOC results file
Writing sheep VOC results file
Writing sofa VOC results file
Writing train VOC results file
Writing tvmonitor VOC results file
VOC07 metric? Yes
AP for aeroplane = 0.0284
AP for bicycle = 0.0014
AP for bird = 0.0223
AP for boat = 0.0064
AP for bottle = 0.0032
AP for bus = 0.0057
AP for car = 0.0306
AP for cat = 0.0362
AP for chair = 0.0029
AP for cow = 0.0455
AP for diningtable = 0.0179
AP for dog = 0.0159
AP for horse = 0.0076
AP for motorbik